<a href="https://colab.research.google.com/github/wayne-chen1887/aps360-project/blob/discriminators-test/discriminators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import os 
import matplotlib.pyplot as plt

In [ ]:
"""
Input: 1920 x 1080

          kernel | stride | output 
conv1 --> 5 x 5  | 2 x 2  | 8  x 958 x 538
conv2 --> 5 x 5  | 2 x 2  | 16 x 477 x 267
conv3 --> 5 x 5  | 2 x 2  | 16 x 237 x 132
conv4 --> 5 x 5  | 2 x 2  | 32 x 117  x 64
conv5 --> 5 x 5  | 2 x 2  | 32 x 57  x 30
conv6 --> 5 x 5  | 2 x 2  | 64 x 27  x 13


          input        | output
fc    --> 64 x 27 x 13 | 1024
"""

In [ ]:
class Global(nn.Module):
  def __init__(self):
    super(Global, self).__init__()
    self.name = "Global"
    self.conv1 = nn.Conv2d(in_channels=3,  out_channels=8, kernel_size=5, stride=2, padding=0)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=2, padding=0)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=2, padding=0)
    self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=0)
    self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=2, padding=0)
    self.conv6 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=2, padding=0)
    self.fc1 = nn.Linear( 64*27*13, 1024)

  def forward(self, x):
    x = F.LeakyReLU(self.conv1(x))
    x = F.LeakyReLU(self.conv2(x))
    x = F.LeakyReLU(self.conv3(x))
    x = F.LeakyReLU(self.conv4(x))
    x = F.LeakyReLU(self.conv5(x))
    x = F.LeakyReLU(self.conv6(x))
    x = x.view(-1, 64*27*13)
    x = self.fc1(x)
    return x

In [ ]:
"""
Input: 160 x 90

          kernel | stride | output 
conv1 --> 5 x 5  | 2 x 2  | 8  x 78 x 43
conv2 --> 5 x 5  | 2 x 2  | 16 x 37 x 20
conv3 --> 5 x 5  | 1 x 1  | 32 x 33 x 16
conv4 --> 5 x 5  | 1 x 1  | 64 x 29  x 12
conv5 --> 5 x 5  | 2 x 2  | 64 x 13  x 4

          input        | output
fc    --> 64 x 13 x 4  | 1024
"""

In [ ]:
class Local(nn.Module):
  def __init__(self):
    super(Local, self).__init__()
    self.name = "Local"
    self.conv1 = nn.Conv2d(in_channels=3,  out_channels=8, kernel_size=5, stride=2, padding=0)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, stride=2, padding=0)
    self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
    self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0)
    self.conv5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=2, padding=0)
    self.fc1 = nn.Linear( 64*13*4, 1024)

  def forward(self, x):
    x = F.LeakyReLU(self.conv1(x))
    x = F.LeakyReLU(self.conv2(x))
    x = F.LeakyReLU(self.conv3(x))
    x = F.LeakyReLU(self.conv4(x))
    x = F.LeakyReLU(self.conv5(x))
    x = x.view(-1, 64*13*4)
    x = self.fc1(x)
    return x

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.name = "Discriminator"
    self.global_n = Global()
    self.local_n = Local()
    self.fc1 = nn.Linear(2048, 1)
  def forward(self, x):
    ld , gd, = x
    ld = self.local_n(ld)
    gd = self.global_n(gd)
    x = torch.cat((ld, gd))
    x = nn.Sigmoid(self.fc1(x))
    return x